In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
import matplotlib.animation as animation
import csv
# matplotlib.style.use('seaborn')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
from wordcloud import WordCloud

from mpl_toolkits.mplot3d import Axes3D

In [2]:
author_data=pd.read_csv("author_data_clusters.csv")
author_data.index=author_data.id
author_data=author_data[["gender","age","topic","sign","grupo_edad","Freq"]]
author_data.sample(20)

,gender,age,topic,sign,grupo_edad,Freq
id,,,,,,
445935,male,26,Agriculture,Pisces,B,233
3927867,male,16,Student,Cancer,A,4
3750679,female,46,indUnk,Libra,C,2
3672318,male,17,Engineering,Aries,A,5
3965275,female,15,indUnk,Capricorn,A,4
3093675,female,14,Student,Sagittarius,A,26
3398614,female,42,indUnk,Libra,C,4
3802222,female,13,Student,Gemini,A,12
946506,female,25,Education,Aquarius,B,7


In [3]:
tfidf_data=pd.read_csv("usr_txt_ftr_3.csv",header=0,index_col="id")#_2# This dataframe is already normalized to max val
# tfidf_data.index=author_data.id
# # author_data=author_data[["gender","age","topic","sign","grupo_edad","Freq"]]
tfidf_data.sample(20)

,00,000,10,100,11,12,13,14,15,16,...,ya,yay,yea,yeah,year,years,yes,yesterday,york,young
id,,,,,,,,,,,,,,,,,,,,,
3865520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3737807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1593902,0.000000,0.000000,0.012989,0.004139,0.007045,0.007333,0.004725,0.003588,0.006942,0.015518,...,0.008044,0.000000,0.000000,0.011708,0.031184,0.010735,0.009455,0.000000,0.0,0.000000
3367290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001967,0.000000,0.002127,0.0,0.000000
3445665,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3407461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1232234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3689779,0.000000,0.010793,0.000000,0.000000,0.007123,0.003613,0.000000,0.016015,0.000000,0.002929,...,0.000000,0.000000,0.000000,0.000000,0.012804,0.001929,0.000000,0.000000,0.0,0.011162
3927641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.006562


In [4]:
author_data=author_data.loc[tfidf_data.index,:]# select sample tfidf matrix /// Cuando la matriz viene de una muestra

In [ ]:
# author_data.Freq=author_data.Freq/author_data.Freq.max() # Normalizar frecuencia de publicaciones

In [ ]:
# enco_cath=pd.get_dummies(author_data[["gender","grupo_edad"]]).astype("float64")

In [ ]:
# enco_cath=enco_cath.loc[:,["gender_female","grupo_edad_A","grupo_edad_B","grupo_edad_C"]]

In [ ]:
# author_data_norm=enco_cath.join(author_data.Freq)
# author_data_norm=author_data_norm.join(tfidf_data)

In [ ]:
# author_data_norm.sample(20)

# Clustering

In [ ]:
# auth_ward_full=hierarchy.linkage(author_data_norm, 'ward')#tfidf_data

In [ ]:
# plt.figure(figsize=(15,35))
# dn_tf_full = hierarchy.dendrogram(auth_ward_full,orientation="left")
# plt.title('Ward Link Dendrogram')
# plt.show()

In [ ]:
# 3d Embedding for visualization
spect_emb_3=manifold.SpectralEmbedding(n_components=3, n_jobs=-1).fit_transform(tfidf_data)#author_data_norm)#

In [ ]:
auth_ward_3=hierarchy.linkage(spect_emb_3, 'ward')

In [ ]:
plt.figure(figsize=(15,35))
dn_tf_3=hierarchy.dendrogram(auth_ward_3,orientation="left")
plt.title('Ward Link Dendrogram from 3d spectral embedding')
plt.show()

In [ ]:
clustering_3d_ward = AgglomerativeClustering(linkage='ward', n_clusters=6)
clustering_3d_ward.fit(spect_emb_3)

In [ ]:
pd.crosstab(index=clustering_3d_ward.labels_, columns="count")

In [ ]:
# spect_emb_3=spect_emb_3*1000

In [ ]:
plt.figure(figsize=(10,30))
par_alf=0.1
plt.subplot(311)
plt.scatter(spect_emb_3[:,0],spect_emb_3[:,1], alpha=par_alf,s=45, c=clustering_3d_ward.labels_, cmap=plt.cm.Set1)
plt.title("2d spectral coordinates from 3d spectral hierarchical (ward) clustering")

plt.subplot(312)
plt.scatter(spect_emb_3[:,2],spect_emb_3[:,1], alpha=par_alf,s=45, c=clustering_3d_ward.labels_, cmap=plt.cm.Set1)
plt.title("2d spectral coordinates from 3d spectral hierarchical (ward) clustering")

plt.subplot(313)
plt.scatter(spect_emb_3[:,0],spect_emb_3[:,2], alpha=par_alf,s=45, c=clustering_3d_ward.labels_, cmap=plt.cm.Set1)
plt.title("2d spectral coordinates from 3d spectral hierarchical (ward) clustering")

In [ ]:
# # Set up formatting for the movie files
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# fig = plt.figure(figsize=(15,15))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(xs=spect_emb_3[:,0],
#            ys=spect_emb_3[:,1],
#            zs=spect_emb_3[:,2],
#            c=clustering_3d_ward.labels_, 
#            alpha=0.35,
#            cmap=plt.cm.Set1)
# plt.title("Ward's hierarchical clustering from spectral embedding" )
# # plt.axis('off') # remove axes for visual appeal

# def rotate(angle):
#     ax.view_init(azim=angle)

# print("Making animation")
# rot_animation = animation.FuncAnimation(fig, rotate, frames=np.arange(0, 362, 2), interval=100)
# print("Saving Movie File")
# rot_animation.save('cluster_anim_2.mp4', writer=writer)
# print("Saving animated GIF File")
# rot_animation.save('whljgr_clustering_2.gif', dpi=80, writer='imagemagick')

In [ ]:
author_data["cluster"]=clustering_3d_ward.labels_
author_data["embd_x_coord"]=spect_emb_3[:,0]
author_data["embd_y_coord"]=spect_emb_3[:,1]
author_data["embd_z_coord"]=spect_emb_3[:,2]

In [ ]:
author_data.head(30)

In [ ]:
author_data.to_csv("author_data_clusters_2.csv")

# Explore Clusters

In [ ]:
pd.crosstab(index=author_data.topic, columns=author_data.cluster)

In [ ]:
pd.crosstab(index=author_data.grupo_edad, columns=author_data.cluster)

In [ ]:
pd.crosstab(index=[author_data.topic,author_data.grupo_edad], columns=author_data.cluster)

In [ ]:
centroides_median=np.array([[np.median(author_data.loc[author_data.cluster==clst,"embd_x_coord"]),
                             np.median(author_data.loc[author_data.cluster==clst,"embd_y_coord"]),
                             np.median(author_data.loc[author_data.cluster==clst,"embd_z_coord"])]
 for clst in list(set(author_data.cluster))])

In [ ]:
enumerate(list(set(author_data.cluster)))

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=spect_emb_3[:,0],
           ys=spect_emb_3[:,1],
           zs=spect_emb_3[:,2],
           c=clustering_3d_ward.labels_, 
           alpha=0.05,
           cmap=plt.cm.Set1)

# ax.scatter(xs=centroides_median[:,0],
#            ys=centroides_median[:,1],
#            zs=centroides_median[:,2], 
#            s=500)

for i, txt in enumerate(list(set(author_data.cluster))):
    ax.text(centroides_median[i,0],
            centroides_median[i,1],
            centroides_median[i,2],
            '%s' % (str(txt)), size=20, zorder=1, color='k')

plt.title("Ward's hierarchical clustering from spectral embedding" )

# Wordclouds

In [ ]:
# tfidf_data=pd.read_csv("df_out.csv",header=0,index_col="id")
# del tfidf_data['Unnamed: 0']
# tfidf_data.sample(20)
# tfidf_data.shape

In [ ]:
author_tfidf=pd.read_csv("usr_txt_ftr_3.csv",header=0,index_col="id")
del author_tfidf['don']
del author_tfidf['just']
print(author_tfidf.shape)
author_tfidf.sample(20)

In [ ]:
author_data=pd.read_csv("author_data_clusters_2.csv",header=0,index_col="id")
print(author_data.shape)
author_data.sample(20)

In [ ]:
clrs_wrdcld=matplotlib.colors.LinearSegmentedColormap.from_list("", [#'#a50026',
                                                                     '#d73027',
                                                                     '#f46d43',
                                                                     '#fdae61',
#                                                                      '#fee090',
#                                                                      '#ffffbf',
#                                                                      '#e0f3f8',
#                                                                      '#abd9e9',
                                                                     '#74add1',
                                                                     '#4575b4',
                                                                     '#313695'])

In [ ]:
pic_wide=900
def wrd_cld_dic_clust(clust_labels,clust_id,top_n=300):
    txt_freqs= author_tfidf[clust_labels==clust_id].sum(axis=0)
    indices = np.argsort(txt_freqs)[::-1]
    features =list(author_tfidf.columns)
    top_features = [[features[i],txt_freqs[i]] for i in indices[:top_n]]
    fr_dic={ str(features[i]).upper():txt_freqs[i] for i in indices[:top_n]}
    wc = WordCloud(width=pic_wide,  height=int(pic_wide*(9/21)), 
               background_color=None, mode="RGBA",min_font_size=8,
               colormap=clrs_wrdcld)#Dark2_r)#Paired_r)#tab10_r)#Set1_r) #
    wc.generate_from_frequencies(fr_dic)
    return wc

In [ ]:
plt.figure(figsize=(16,9))
plt.imshow(wrd_cld_dic_clust(author_data.cluster,0,200), interpolation='bilinear')
plt.axis("off")
#plt.savefig('wrdcld_1.png')

In [ ]:
plt.figure(figsize=(16,9))
plt.imshow(wrd_cld_dic_clust(author_data.cluster,1,200), interpolation='bilinear')
plt.axis("off")
#plt.savefig('wrdcld_1.png')

In [ ]:
plt.figure(figsize=(16,9))
plt.imshow(wrd_cld_dic_clust(author_data.cluster,2,200), interpolation='bilinear')
plt.axis("off")
#plt.savefig('wrdcld_1.png')

In [ ]:
plt.figure(figsize=(16,9))
plt.imshow(wrd_cld_dic_clust(author_data.cluster,3,200), interpolation='bilinear')
plt.axis("off")
#plt.savefig('wrdcld_1.png')

In [ ]:
plt.figure(figsize=(16,9))
plt.imshow(wrd_cld_dic_clust(author_data.cluster,4,200), interpolation='bilinear')
plt.axis("off")
#plt.savefig('wrdcld_1.png')

In [ ]:
plt.figure(figsize=(16,9))
plt.imshow(wrd_cld_dic_clust(author_data.cluster,5,200), interpolation='bilinear')
plt.axis("off")
#plt.savefig('wrdcld_1.png')